In [1]:
import os

if "/Users/felipeforero/Dev/PokemonOpti" != os.getcwd():
    os.chdir("..")

In [2]:
os.getcwd()

'/Users/felipeforero/Dev/PokemonOpti'

In [3]:
import pickle
import pandas as pd
from scipy.stats import f_oneway
from poketactician.objectives import attack_obj_fun, team_coverage_fun, self_coverage_fun
import plotly.graph_objects as go
import numpy as np

In [4]:
#Objective Functions
attackObjFun = lambda team: attack_obj_fun(team)
teamCoverageFun = lambda team: team_coverage_fun(team)
selfCoverageFun = lambda team: self_coverage_fun(team)
objectiveFuncs = [
    attackObjFun,
    # teamCoverageFun,
]

In [5]:
# Import experiment results
with open('experiments/data/averages_dict_complete.pkl', 'rb') as file:
    averages_dict = pickle.load(file)

In [ ]:
averages_dict[0.1][0]

<class 'list'>


<class 'list'>


In [ ]:
# Dict Key -> MultiProc Iteration -> Optimizer Iteration -> the list of that iteration -> CandSetIter -> Team
# averages_dict["0.1"][0][0][9][0][-1]
averages_dict[0.1][0][0][-1][-1]

array([[422,  25,   8,   9,  10],
       [509,  15,  16,   4,  10],
       [238,  16,  17,   8,  10],
       [290,  16,  15,   0,  14],
       [  5,   7,   3,  28,  18],
       [276,  17,   2,  10,   0]])

# Test if ρ has significance on best team value

In [10]:
populations_transformed = {
    key: [attackObjFun(data[i][0][-1][-1]) for i in range(0, len(data))]
    for key, data in averages_dict.items()
}

In [11]:
f_statistic, p_value = f_oneway(*list(populations_transformed.values()))

In [12]:
# Print the ANOVA results
print("ANOVA Results:")
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# Determine whether to reject the null hypothesis based on the p-value
alpha = 0.05
if p_value < alpha:
    print(
        "Reject the null hypothesis. There are significant differences between the populations."
    )
else:
    print(
        "Fail to reject the null hypothesis. There is no significant difference between the populations."
    )

ANOVA Results:
F-statistic: 1.1922971453654487
P-value: 0.2916925658045562
Fail to reject the null hypothesis. There is no significant difference between the populations.


In [13]:
fig = go.Figure()
for key, averages in populations_transformed.items():
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(averages)),
            y=averages,
            mode="lines",
            name=f"ρ: {key}",
        )
    )
fig.update_layout(
    xaxis_title="Iteration",
    yaxis_title="Objective Function Value",
    title=f"Analyzing ρ Variation",
)
fig.show()

In [14]:
pop_averages = {
    key: np.average(value) for key, value in populations_transformed.items()
}
pop_st_dev = {key: np.std(value) for key, value in populations_transformed.items()}
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(0.1, 1.2, 0.1),
        y=list(pop_averages.values()),
        error_y=dict(type="data", array=list(pop_st_dev.values()), visible=True),
        mode="lines",
        name=f"ρ: {key}",
    )
)
fig.update_layout(
    xaxis_title="ρ",
    yaxis_title="Objective Function Value",
    title=f"Analyzing ρ Variation (Mean and St. Dev.)",
)
fig.show()

# Test if ρ has significance on convergence iteration

In [15]:
def find_stabilization_index(values, threshold=0.001):
    """
    Find the index at which the values stabilize.

    Parameters:
    - values: List or array of numeric values.
    - threshold: The relative difference threshold to determine stabilization.

    Returns:
    - The index at which the values stabilize.
    """
    for i in range(1, len(values)):
        relative_difference = abs((values[i] - values[i - 1]) / values[i - 1])
        if relative_difference < threshold:
            return i
    # If no stabilization is found, return the last index
    return len(values) - 1

In [16]:
populations_conv = {
    key: [find_stabilization_index([attackObjFun(data[i][j][-1][-1]) for j in range(0,100)]) for i in range(0, len(data))]
    for key, data in averages_dict.items()
}

In [17]:
f_statistic, p_value = f_oneway(*list(populations_conv.values()))

In [18]:
# Print the ANOVA results
print("ANOVA Results:")
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# Determine whether to reject the null hypothesis based on the p-value
alpha = 0.05
if p_value < alpha:
    print(
        "Reject the null hypothesis. There are significant differences between the populations."
    )
else:
    print(
        "Fail to reject the null hypothesis. There is no significant difference between the populations."
    )

ANOVA Results:
F-statistic: 555.547853406346
P-value: 0.0
Reject the null hypothesis. There are significant differences between the populations.


In [19]:
fig = go.Figure()
for key, averages in populations_conv.items():
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(averages)),
            y=averages,
            mode="lines",
            name=f"ρ: {key}",
        )
    )
fig.update_layout(
    xaxis_title="Iteration",
    yaxis_title="Index of Convergence",
    title=f"Analyzing ρ Variation",
)
fig.show()

In [20]:
pop_averages = {key: np.average(value) for key, value in populations_conv.items()}
pop_st_dev = {key: np.std(value) for key, value in populations_conv.items()}
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(0, 1.2, 0.1),
        y=list(pop_averages.values()),
        error_y=dict(type="data", array=list(pop_st_dev.values()), visible=True),
        mode="lines",
        name=f"ρ: {key}",
    )
)
fig.update_layout(
    xaxis_title="ρ",
    yaxis_title="Index of Convergence",
    title=f"Analyzing ρ Variation (Mean and St. Dev.)",
)
fig.show()

In [21]:
pop_averages = {key: np.average(value) for key, value in populations_conv.items()}
pop_st_dev = {key: np.std(value) for key, value in populations_conv.items()}
fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=np.arange(0.1, 1, 0.1),
#         y=list(pop_averages.values()),
#         error_y=dict(type="data", array=list(pop_st_dev.values()), visible=True),
#         mode="lines",
#         name=f"ρ: {key}",
#     )
# )
for key, values in populations_conv.items():
    fig.add_trace(
        go.Box(y=values, name=key, notched=True)
    )
fig.update_layout(
    xaxis_title="ρ",
    yaxis_title="Index of Convergence",
    title=f"Analyzing ρ Variation (Mean and St. Dev.)",
)
fig.show()

# Plot Iterations

In [22]:
# Dict Key -> MultiProc Iteration -> Optimizer Iteration -> the list of that iteration -> CandSetIter -> Team
# averages_dict[1.1][0][0][-1][-1]
averrages_1 = averages_dict[1]
populations_obj_vale = [
    [attack_obj_fun(team[0][-1]) for team in iteration] for iteration in averrages_1
]
fig = go.Figure()
for i, averages in enumerate(populations_obj_vale):
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(averages)),
            y=averages,
            mode="lines",
            name=f"ρ: {i}",
        )
    )
fig.update_layout(
    xaxis_title="Iteration",
    yaxis_title="Objective Function Value",
    title=f"Analyzing ρ Variation",
)
fig.show()